In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
from nbdev.showdoc import show_doc
from getpass import getuser, getpass
from fiscaliza.redmine import detalhar_inspecao, validar_dicionario, relatar_inspecao
from rich.console import Console
console = Console()

# Fiscaliza

> Scripts para baixar informações sobre inspeções no Sistema Fiscaliza da Anatel e atualizar inspeções por meio da API do Redmine.

## Instalação

`pip install fiscaliza`

## Como utilizar

O `Fiscaliza` da Anatel é um sistema web que utiliza a plataforma Redmine. É basicamente um softaware de gerenciamento e atividades.

O Fiscaliza possui 2 módulos para os fiscais utilizarem:
* Módulo de Teste ( Homologação ) : "https://sistemashm.anatel.gov.br/fiscaliza"
* Módulo de Produção: "https://sistemas.anatel.gov.br/fiscaliza/"

Ambos módulos precisam de um usuário e senha com acesso autorizado, em geral somente servidores da Anatel.

O Escopo desse módulo basicamente está encapsulado em 3 funções básicas:
* detalhar_inspecao - **Retorna as informações do estado atual da Issue ( Inspeção )**
* validar_dicionario - **Para dado dicionário de dados, formata-o para a API do Redmine**
* relatar_inspecao - **Atualiza e Issue com o dicionário de dados passado**

> Note: O escopo da terceira função `relatar_inspecao` possui escopo limitado. Atualmente somente é formatado e relatado Inspeções ( Issue ) to tipo "Uso do Espectro - Monitoração". Essa inspeção é a principal utilizada para as atividades de monitoração da Anatel e foi o que motivou a criação da presente biblioteca. Outras inspeções possuem campos distintos e assim exigem formatação distinta. Versões futuras poderão abarcar o relato de diferentes inspeções. 

In [ ]:
#hide
show_doc(detalhar_inspecao)

<h4 id="detalhar_inspecao" class="doc_header"><code>detalhar_inspecao</code><a href="https://github.com/ronaldokun/fiscaliza/tree/master/fiscaliza/redmine.py#L425" class="source_link" style="float:right">[source]</a></h4>

> <code>detalhar_inspecao</code>(**`inspecao`**:"Número da Inspeção a ser relatada", **`login`**:"Login Anatel do Usuário"=*`None`*, **`senha`**:"Senha Utilizada nos Sistemas Interativos da Anatel"=*`None`*, **`fiscaliza`**:"Objeto Redmine logado, opcional ao login e senha"=*`None`*, **`teste`**:"Indica se o relato será de teste"=*`True`*)

Recebe número da inspeção `inspecao`, o login e senha ou opcionalmente objeto Redmine logado `fiscaliza`
inspecao: str - Número da Inspeção a ser relatada
login: str - Login Anatel do Usuário
senha: str - Senha Utilizada nos Sistemas Interativos da
fiscaliza: Redmine - Objeto Redmine logado, opcional ao login e senha
teste: bool - Caso verdadeiro o Fiscaliza de Teste ( Homologação ) é utilizado

Returns:
    dict: Retorna um dicionário com a Situação Atual e campos preenchidos da Inspeção

Vamos exemplificar a Inspeção de Teste `53689`

In [ ]:
login = getuser()
senha = getpass()
inspecao = '57689'

 ··········


In [ ]:
inspecao = '51849'
detalhes = detalhar_inspecao(inspecao, login, senha, teste=False)

In [ ]:
for k,v in detalhes.items():
    console.print(f'[blue]{k} [red]-> [green]{v}')

id -> 51849

subject -> INSP_GR01_2021_0456

status -> Rascunho

priority -> Normal

start_date -> 2021-03-09

...

id_ACAO -> 47823

nome_ACAO -> ACAO_GR01_2021_0338

descricao_ACAO -> [PMEC 2021] Monitorar canais e faixas de frequências relacionados às 
aplicações críticas (como, por exemplo, radionavegação e radiocomunicação aeronáutica e 
canais de emergência) na forma a ser estabelecida no Plano de Ação de Fiscalização GR08FI2 
(SEI nº 6383328).

Users -> ['Alexandre Elias de Andrade Oliveira', 'Alexandre Freitas de Lima', 'Alexandre 
Inacio', 'Alexandre Junzo Hamada', 'Alfredo de Andrade Filho', 'Ananias Pereira', 'Antonio 
Carlos Cardoso de Mello', 'Aparecido Sebastiao da Silva', 'Arthur Pisaruk', 'Carlos Augusto 
de Carvalho', 'Carlos Eduardo Guimaraes Silveira', 'Carlos da Paixao Filho', 'Celio Yukio 
Takahashi', 'Celso Luiz Maximino', 'Diogo Caldeira', 'Ediceu Beraldi', 'Eduardo Narkevicius',
'Elcio Maehara', 'Eustaquio Lages Duarte', 'Fiscal UD', 'Gauber Albuquerque', 'Gilson Ponce 
Ayres Filho', 'Helio Lopes de Carvalho Filho', 'Higor da Paz Melo', 'Hugo Santana Lima', 
'Humberto Barbosa Vinagre', 'Jamilson Evangelista', 'Joao Yokoyama', 'Joaquim Miranda', 'José
Antônio S. Sanches', 'Julio Cesar de Assis Santos', 'Kiyotomo Kawamura', 'Laert Calil 
Junior', 'Lannei Vilela Moraes', 'Luis Lagomes', 'Luiz Vinicios Mielniczuk Seelig', 'Marcelo 
Augusto Scacabarozi', 'Marcelo Vaz Netto', 'Marcio Colazingari', 'Marcio Costa', 'Marcio 
Rodrigues Maciel', 'Marcos Antônio Rodrigues', 'Marcos Juliano Valim da Silva', 'Maria Teresa
Flosi Garrafa', 'Mario Augusto Volpini', 'Murilo Amaro', 'Osnir Lopes', 'Paulo Diogo Costa', 
'Pedro Arai', 'Renato Sadao Kushioyada', 'Ricardo Santos Marques', 'Ricardo da Silva e 
Souza', 'Roberto Carlos Soares Campos', 'Roberto Ferreira dos Santos', 'Roberto Takata', 
'Rodrigo Barbosa de Paula', 'Rogerio Zambotto', 'Ronaldo da Silva Alves Batista', 'Sergio 
Pereira', 'Thiago Silva', 'Thomaz Honma Ishida', 'Vinicius Paiva de Oliveira', 'Vitor 
Zelada', 'Wellington Devechi Piauilino', 'Wladimir Senise']

Modificado -> Atualizado por Ronaldo da Silva Alves Batista em 2021-05-24 às 13:12:32

In [ ]:
inspecao = '57690'
detalhes = detalhar_inspecao(inspecao, login, senha)

In [ ]:
for k,v in detalhes.items():
    console.print(f'[red]{k} [blue]-> [green]{v}')

id -> 57690

subject -> INSP_GR01_2021_0509

status -> Rascunho

priority -> Normal

start_date -> 

...

id_ACAO -> 52876

nome_ACAO -> ACAO_GR01_2021_0491

descricao_ACAO -> Teste

Users -> ['Alexandre Elias de Andrade Oliveira', 'Alexandre Freitas de Lima', 'Alexandre 
Inacio', 'Alexandre Junzo Hamada', 'Alfredo de Andrade Filho', 'Ananias Pereira', 'Antonio 
Carlos Cardoso de Mello', 'Aparecido Sebastiao da Silva', 'Arthur Pisaruk', 'Carlos Augusto 
de Carvalho', 'Carlos Eduardo Guimaraes Silveira', 'Carlos da Paixao Filho', 'Celio Yukio 
Takahashi', 'Celso Luiz Maximino', 'Diogo Caldeira', 'Ediceu Beraldi', 'Eduardo Narkevicius',
'Elcio Maehara', 'Eustaquio Lages Duarte', 'Fiscal UD', 'Gauber Albuquerque', 'Gilson Ponce 
Ayres Filho', 'Helio Lopes de Carvalho Filho', 'Higor Paz Melo', 'Hugo Santana Lima', 
'Humberto Barbosa Vinagre', 'Jamilson Evangelista', 'Joao Yokoyama', 'Joaquim Miranda', 'José
Antônio S. Sanches', 'Julio Cesar de Assis Santos', 'Kiyotomo Kawamura', 'Laert Calil 
Junior', 'Lannei Vilela Moraes', 'Luis Lagomes', 'Luiz Vinicios Mielniczuk Seelig', 'Marcelo 
Augusto Scacabarozi', 'Marcio Costa', 'Marcio Rodrigues Maciel', 'Marcos Antônio Rodrigues', 
'Marcos Juliano Valim da Silva', 'Maria Teresa Flosi Garrafa', 'Mario Augusto Volpini', 
'Murilo Amaro', 'Osnir Lopes', 'Paulo Diogo Costa', 'Pedro Arai', 'Renato Sadao Kushioyada', 
'Ricardo Santos Marques', 'Ricardo da Silva e Souza', 'Roberto Carlos Soares Campos', 
'Roberto Ferreira dos Santos', 'Roberto Takata', 'Rodrigo Barbosa de Paula', 'Rogerio 
Zambotto', 'Ronaldo da Silva Alves Batista', 'Sergio Pereira', 'Thiago Silva', 'Thomaz Honma 
Ishida', 'Vinicius Paiva de Oliveira', 'Vitor Zelada', 'Wellington Devechi Piauilino', 
'Wladimir Senise', 'marcio colazingari']

Modificado -> Atualizado por Ronaldo da Silva Alves Batista em 2021-05-24 às 12:56:59

Agora vamos testar o retorno de informações para um Inspeção no Fiscaliza de produção `teste=False`.

In [ ]:
f = auth_user(login, senha)

In [ ]:
issue = f.issue.get(inspecao, include=["relations", "attachments"])

In [ ]:
list(issue)

[('relations',
  [{'id': 20663,
    'issue_id': 57690,
    'issue_to_id': 52876,
    'relation_type': 'blocks',
    'delay': None}]),
 ('time_entries', None),
 ('children', None),
 ('attachments', []),
 ('changesets', None),
 ('journals', None),
 ('watchers', None),
 ('id', 57690),
 ('project', {'id': 25, 'name': 'GR01'}),
 ('tracker', {'id': 1, 'name': 'Inspeção (Objeto)'}),
 ('status', {'id': 1, 'name': 'Rascunho'}),
 ('priority', {'id': 2, 'name': 'Normal'}),
 ('author', {'id': 887, 'name': 'Ronaldo da Silva Alves Batista'}),
 ('subject', 'INSP_GR01_2021_0509'),
 ('done_ratio', 0),
 ('custom_fields',
  [{'id': 89, 'name': 'Classe da Inspeção'},
   {'id': 2, 'name': 'Tipo de inspeção'},
   {'id': 5, 'name': 'Ano de Execução', 'value': '2021'},
   {'id': 422,
    'name': 'Nº SEI do Processo',
    'value': '{"numero"=>"53504.000007/2021-50", "link_acesso"=>"https://seihm.anatel.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1962455"}'},
   {'id': 22, 'name': 'De

In [ ]:
from datetime import datetime, timezone, timedelta

date = datetime.fromisoformat('2021-05-24T15:56:59') - timedelta(hours=3)
date

datetime.datetime(2021, 5, 24, 12, 56, 59)

In [ ]:
date = datetime.strptime('2021-05-24T15:56:59', '%Y-%m-%dT%H:%M:%S', timezone(timedelta(hours=-3)))

TypeError: strptime() takes exactly 2 arguments (3 given)

## Dicionário de Dados
A função anterior somente retorna as informações já constantes em dada Inspeção ( Issue ) do Fiscaliza. Para alterarmos ou atualizarmos dada inspeção, precisamos passar um dicionário de dados ou um caminho para um arquivo `.json` ou pickle `.pkl` onde conste esse dicionário de dados serializado. O exemplo seguinte mostra um dicionário de dados com as informações básicas constantes de uma monitoração e a formatação que elas são validadas:

In [ ]:
d = {}

d['Classe da Inspeção'] = 'Técnica' # str

d['Tipo de inspeção'] = 'Uso do Espectro - Monitoração' #str

d['Descrição da Inspeção'] = '''Atendimento da Denúncia AC202010213075425 (6104512), 
verificação da Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e seus harmônicos, 
além da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 750MHz.''' #str

d['Fiscal Responsável'] = 'Ronaldo da Silva Alves Batista' #str

d['Fiscais'] = ['Ronaldo da Silva Alves Batista', 'Paulo Diogo Costa', 'Mario Augusto Volpini'] #string ou lista de strings

# Windows
d['Html'] = 'D:\\OneDrive - ANATEL\\Monitoramento\\Processos\\53504.0005432021-55\\Guarulhos.html' #str

#ou 

#Unix d['Html'] = '/d/OneDrive - ANATEL/Monitoramento/53504.0005432021-55/Guarulhos.html' #str
            
d['Gerar Relatório'] = 1 # int 0 ou 1

d['Frequência Inicial']  = 54 #int ou float

d['Unidade da Frequência Inicial'] = 'MHz' #string

d['Frequência Final'] = 700 #int ou float

d['Unidade da Frequência Final'] = 'MHz' #string

d['Data de Início'] = '2021-03-19' #YYYY-MM-DD #string nesse formato

d['Data Limite'] = '2021-12-31'  #YYYY-MM-DD #string nesse formato

d['UF/Município'] = "SP/São Paulo" # string ou Lista de Strings: ["SP/São Paulo", "SP/Sorocaba"]

d['Serviços da Inspeção'] = ['230', '231', '800'] # String ou Lista de Strings

d['Qnt. de emissões na faixa'] = 12 # int

d['Emissões não autorizadas/desc'] = 70 # int

d['Horas de Preparação'] = 2 # int

d['Horas de Deslocamento'] = 0 # int

d['Horas de Execução'] = 32 # int

d['Horas de Conclusão'] = 6 # int

d['Latitude (coordenadas)'] = -22.94694 # float

d['Longitude (coordenadas)'] = -43.21944 # float

d['Uso de PF'] = 'Não se aplica PF - uso apenas de formulários' # string

d['Ação de risco à vida criada?'] = 'Não' # string Sim | Não

d['Impossibilidade acesso online?'] = '0' # string '0' | '1'

d['Reservar Instrumentos?'] = '0' #string '0' = Não | '1' = 'Sim'

d['Utilizou algum instrumento?'] = '0'

#d['Reserva de Instrumentos'] = '' 

d['Notes'] = "Não foi constatada irregularidade no Período monitorado" # string

# No caso de uma tabela, string formatada como csv

d['Notes'] = """Faixa, Classe Especial, Classe A, Classe B, Classe C
                VHF-L,0,5,7,5
                VHF-H,0,12,1,0
                UHF,1,1,2,4
                FM,5,1,0,0
                RADCOM,0,0,0,0
                Outorgadas com indícios de irregularidades,1,2,3,4
            """

A API do Redmine possui formatos específicos de como esses campos devem ser submetidos, validar os formatos acima e fazer essa formatação exigida pela API do Redmine é o papel da função `validar_dicionario`.

In [ ]:
#hide
show_doc(validar_dicionario)

<h4 id="validar_dicionario" class="doc_header"><code>validar_dicionario</code><a href="https://github.com/ronaldokun/fiscaliza/tree/main/fiscaliza/redmine.py#L57" class="source_link" style="float:right">[source]</a></h4>

> <code>validar_dicionario</code>(**`data_dict`**:"Dicionário de Dados ou Caminho para o arquivo .json", **`inspecao`**:"Número da Inspeção a ser relatada", **`login`**:"Login Anatel do Usuário"=*`None`*, **`senha`**:"Senha Utilizada nos Sistemas Interativos da Anatel"=*`None`*, **`fiscaliza`**:"Objeto Redmine logado, opcional ao login e senha"=*`None`*, **`teste`**:"Caso verdadeiro o Fiscaliza de Teste ( Homologação ) é utilizado"=*`True`*, **`save_path`**:"Caminho para salvar o dicionário formatado"=*`None`*)



In [ ]:
inspecao = '57689'
dados = validar_dicionario('files/data.json', inspecao, login, senha)

In [ ]:
for k,v in dados.items():
    if k == "Html":
        v = str(v)
        v = v[:100] + '\n...\n' + v[-100:]
    console.print(f'[blue]{k} [red]-> [green]{v}')

Classe_da_Inspecao -> {'id': 89, 'value': '{"valor":"Técnica","texto":"Técnica"}'}

Tipo_de_Inspecao -> {'id': 2, 'value': '{"valor":"Uso do Espectro - Monitoração","texto":"Uso
do Espectro - Monitoração"}'}

Descricao_da_Inspecao -> {'id': 22, 'value': 'Atendimento da Denúncia AC202010213075425 
(6104512), \nverificação da Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e
seus harmônicos, \nalém da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 
750MHz.'}

Fiscal_Responsavel -> {'id': 25, 'value': 'Ronaldo da Silva Alves Batista'}

Fiscais -> {'id': 26, 'value': ['Ronaldo da Silva Alves Batista', 'Paulo Diogo Costa', 'Mario
Augusto Volpini']}

...

Acao_de_risco_a_vida_criada -> {'id': 154, 'value': 'Não'}

Impossibilidade_acesso_online -> {'id': 450, 'value': '0'}

Reservar_Instrumentos -> {'id': 658, 'value': '0'}

Utilizou_algum_instrumento -> {'id': 660, 'value': '0'}

Notes -> |_.  Faixa                                      |_.   Classe Especial |_.   Classe A
|_.   Classe B |_.   Classe C |
|<. VHF-L                                       |>.                 0 |>.          5 |>.     
7 |>.          5 |
|<. VHF-H                                       |>.                 0 |>.         12 |>.     
1 |>.          0 |
|<. UHF                                         |>.                 1 |>.          1 |>.     
2 |>.          4 |
|<. FM                                          |>.                 5 |>.          1 |>.     
0 |>.          0 |
|<. RADCOM                                      |>.                 0 |>.          0 |>.     
0 |>.          0 |
|<. Outorgadas com indícios de irregularidades  |>.                 1 |>.          2 |>.     
3 |>.          4 |

## Relatar Inspeção
A função a seguir é a mais importante do módulo porque ela de fato altera os dados na plataforma Fiscaliza.

In [ ]:
#hide
show_doc(relatar_inspecao)

<h4 id="relatar_inspecao" class="doc_header"><code>relatar_inspecao</code><a href="https://github.com/ronaldokun/fiscaliza/tree/main/fiscaliza/redmine.py#L499" class="source_link" style="float:right">[source]</a></h4>

> <code>relatar_inspecao</code>(**`inspecao`**:"Número da Inspeção a ser relatada", **`login`**:"Login Anatel do Usuário", **`senha`**:"Senha Utilizada nos Sistemas Interativos da Anatel", **`dados`**:"Dicionário já validado com os Dados a serem relatados", **`teste`**:"Indica se o relato será de teste"=*`True`*)

Relata a inspeção `inspecao` com os dados constantes no dicionário `dados`

In [ ]:
inspecao = '57689'
relatar_inspecao(inspecao, login, senha, dados='files/dados.json', teste=True)

Usuário Autenticado com Sucesso 👍

Inspeção 57689 vinculada à Ação {'id_ACAO': 52876, 'nome_ACAO': 'ACAO_GR01_2021_0491', 
'descricao_ACAO': 'Teste'}

Output()

Estado Atual: Rascunho ❗

Output()

...

Sucesso ✨

Output()

Estado Atual: Relatando ❗

Assine o Relatorio_de_Monitoramento: {"numero"=>"0190599", "link_acesso"=>"https://seihm.anat
el.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1962455&id_document
o=1962869"} e chame a função novamente ❗

{'id': '57689',
 'subject': 'INSP_GR01_2021_0508',
 'status': 'Relatando',
 'priority': 'Normal',
 'start_date': '2021-03-19',
 'due_date': '2021-12-31',
 'Classe da Inspeção': 'Técnica',
 'Tipo de inspeção': 'Uso do Espectro - Monitoração',
 'Ano': '2021',
 'Numero_Sei_do_Processo': '{"numero"=>"53504.000007/2021-50", "link_acesso"=>"https://seihm.anatel.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1962455"}',
 'Descrição da Inspeção': 'Atendimento da Denúncia AC202010213075425 (6104512), \nverificação da Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e seus harmônicos, \nalém da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 750MHz.',
 'Fiscal Responsável': 'Ronaldo da Silva Alves Batista',
 'Fiscais': ['Mario Augusto Volpini',
  'Paulo Diogo Costa',
  'Ronaldo da Silva Alves Batista'],
 'Entidade da Inspeção': [],
 'UF/Município': [],
 'Serviços da Inspeção': [],
 'Qnt. de emissões na faixa': '',
 'Emissões não autorizadas

O dicionário de dados possui os dados completos obrigatórios para o relato, no entanto o relato é feito em algumas etapas, utilizando somente algumas das informações do dicionário:
* `Rascunho` para `Aguardando Execução`
* `Aguardando Execução` para `Em Andamento`
* `Em Andamento` para `Relatando`
* `Relatando` para `Relatada`

Na terceira etapa é gerado um relatório da monitoração no Sistema Eletrônico de Informações - SEI. Para executar a quarta etapa é preciso que esse relatório da terceira etapa esteja assinado. Por essa razão a função para na terceira etapa e informa da necessidade do relatório estar assinado.

Após o relatório ser assinado basta chamar a função com os mesmos argumentos para que a etapa final seja realizada.

In [ ]:
estado = relatar_inspecao(inspecao, login, senha, dados=dados, teste=True)

Usuário Autenticado com Sucesso 👍

Inspeção 57689 vinculada à Ação {'id_ACAO': 52876, 'nome_ACAO': 'ACAO_GR01_2021_0491', 
'descricao_ACAO': 'Teste'}

Output()

Estado Atual: Relatando ❗

Output()

Sucesso ✨

Output()

{
    'id': '57689',
    'subject': 'INSP_GR01_2021_0508',
    'status': 'Relatada',
    'priority': 'Normal',
    'start_date': '2021-03-19',
    'due_date': '2021-12-31',
    'Classe da Inspeção': 'Técnica',
    'Tipo de inspeção': 'Uso do Espectro - Monitoração',
    'Ano': '2021',
    'Numero_Sei_do_Processo': '{"numero"=>"53504.000007/2021-50", "link_acesso"=>"https://sei
hm.anatel.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1962455"}',
    'Descrição da Inspeção': 'Atendimento da Denúncia AC202010213075425 (6104512), 
\nverificação da Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e seus 
harmônicos, \nalém da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 
750MHz.',
    'Fiscal Responsável': 'Ronaldo da Silva Alves Batista',
    'Fiscais': [
        'Ronaldo da Silva Alves Batista',
        'Mario Augusto Volpini',
        'Paulo Diogo Costa'
    ],
    'Entidade da Inspeção': [],
    'UF/Município': ['SP/São Paulo'],
    'Serviços da Inspeção': [
        '800 - COLETIVO - RETRANSMISSAO DE T.V.',
        '231 - COLETIVO - RADIODIFUSÃO COMUNITÁRIA',
        '230 - COLETIVO - RADIODIFUSÃO SONORA EM FREQÜÊNCIA MODULADA'
    ],
    'Qnt. de emissões na faixa': '12',
    'Emissões não autorizadas/desc': '70',
    'Horas de Preparação': '2',
    'Horas de Deslocamento': '0',
    'Horas de Execução': '32',
    'Horas de Conclusão': '6',
    'SAV': '',
    'PCDP': '',
    'Procedimentos': [],
    'Latitude (coordenadas)': '-22.94694',
    'Longitude (coordenadas)': '-43.21944',
    'Uso de PF': 'Não se aplica PF - uso apenas de formulários',
    'Ação de risco à vida criada?': 'Não',
    'Frequência Inicial': '54',
    'Unidade da Frequência Inicial': 'MHz',
    'Frequência Final': '700',
    'Unidade da Frequência Final': 'MHz',
    'Agrupamento': '',
    'AppFiscaliza': '0',
    'Relatorio_de_Monitoramento': '{"numero"=>"0190599", "link_acesso"=>"https://seihm.anatel
.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1962455&id_documento=
1962869"}',
    'Reservar Instrumentos?': '0',
    'Utilizou algum instrumento?': '0',
    'id_ACAO': 52876,
    'nome_ACAO': 'ACAO_GR01_2021_0491',
    'descricao_ACAO': 'Teste',
    'Users': [
        'Alexandre Elias de Andrade Oliveira',
        'Alexandre Freitas de Lima',
        'Alexandre Inacio',
        'Alexandre Junzo Hamada',
        'Alfredo de Andrade Filho',
        'Ananias Pereira',
        'Antonio Carlos Cardoso de Mello',
        'Aparecido Sebastiao da Silva',
        'Arthur Pisaruk',
        'Carlos Augusto de Carvalho',
        'Carlos Eduardo Guimaraes Silveira',
        'Carlos da Paixao Filho',
        'Celio Yukio Takahashi',
        'Celso Luiz Maximino',
        'Diogo Caldeira',
        'Ediceu Beraldi',
        'Eduardo Narkevicius',
        'Elcio Maehara',
        'Eustaquio Lages Duarte',
        'Fiscal UD',
        'Gauber Albuquerque',
        'Gilson Ponce Ayres Filho',
        'Helio Lopes de Carvalho Filho',
        'Higor Paz Melo',
        'Hugo Santana Lima',
        'Humberto Barbosa Vinagre',
        'Jamilson Evangelista',
        'Joao Yokoyama',
        'Joaquim Miranda',
        'José Antônio S. Sanches',
        'Julio Cesar de Assis Santos',
        'Kiyotomo Kawamura',
        'Laert Calil Junior',
        'Lannei Vilela Moraes',
        'Luis Lagomes',
        'Luiz Vinicios Mielniczuk Seelig',
        'Marcelo Augusto Scacabarozi',
        'Marcio Costa',
        'Marcio Rodrigues Maciel',
        'Marcos Antônio Rodrigues',
        'Marcos Juliano Valim da Silva',
        'Maria Teresa Flosi Garrafa',
        'Mario Augusto Volpini',
        'Murilo Amaro',
        'Osnir Lopes',
        'Paulo Diogo Costa',
        'Pedro Arai',
        'Renato Sadao Kushioyada',
        'Ricardo Santos Marques',
        'Ricardo da Silva e Souza',
        'Roberto Carlos Soares Campos',
        'Roberto Ferreira dos Santos',
        'Roberto Takata',
        'Rodrigo B

Inspeção Relatada 😎